In [2]:
import pandas as pd

In [ ]:
# full_raw = pd.read_csv('/Users/javier/VSCODE/datahub/enron_data_fraud_labeled.csv') 

# filtered_raw = full_raw['Body'].iloc[:100_000]

# filtered_raw.to_csv('test.csv')

/var/folders/c9/rl6pcdvd1_xb6ng3n8l9wr2w0000gn/T/ipykernel_84131/2062274805.py:1: DtypeWarning: Columns (7,13) have mixed types. Specify dtype option on import or set low_memory=False.
  full_raw = pd.read_csv('/Users/javier/VSCODE/datahub/enron_data_fraud_labeled.csv')


In [ ]:
filtered_raw = pd.read_csv('local/DSA4213_vsc/final_project/test.csv')

